In [1]:
import mlrose_hiive
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import mlrose_hiive as mlrose
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mlrose_hiive import MaxKColorGenerator, QueensGenerator, FlipFlopGenerator,\
      TSPGenerator, KnapsackGenerator, ContinuousPeaksGenerator
from mlrose_hiive import SARunner, GARunner, NNGSRunner, MIMICRunner, RHCRunner
from mlrose_hiive import SKMLPRunner

In [ ]:
mlrose_hiive.NeuralNetwork(hidden_nodes = [], activation = 'sigmoid', 
                                    algorithm = 'random_hill_climb', 
                                    max_iters = 1000, bias = True, is_classifier = True, 
                                    learning_rate = 0.01, early_stopping = True, 
                                    clip_max = 5, max_attempts = 100, random_state = 3)

In [5]:
flip_problem = FlipFlopGenerator().generate(seed=123456)
queen_problem = QueensGenerator().generate(seed=123456)
knap_problem = KnapsackGenerator().generate(seed=123456)

In [ ]:
# Define decay schedule
schedule = mlrose.ExpDecay()

# Define initial state
init_state = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# Solve problem using simulated annealing
best_state, best_fitness, curve = mlrose.simulated_annealing(
    knap_problem, schedule = schedule,
    max_attempts = 10, max_iters = 1000,
    init_state = init_state, random_state = 123321,
    curve=True
    )

print(best_state)
print(best_fitness)
fig, ax = plt.subplots()
ax.plot(curve[:,1], curve[:,0])
plt.show()

In [7]:
# create a runner class and solve the problem
rhc_run = RHCRunner(problem=knap_problem,
              experiment_name='queens8_sa',
              output_directory=None, # note: specify an output directory to have results saved to disk
              seed=123456,
              iteration_list=2 ** np.arange(11),
              max_attempts=500,
              restart_list=[10, 20]
              )

# the two data frames will contain the results
df_run_stats, df_run_curves = rhc_run.run()

In [10]:
df_run_curves[(df_run_curves['Fitness']==0) & (df_run_curves['Iteration']<20)]

,Iteration,Time,Fitness,FEvals,Restarts,max_iters,current_restart
4980,1,0.002478,0.0,5036.0,10,1024,9
11132,1,0.002478,0.0,5036.0,20,1024,9


In [11]:
# create a runner class and solve the problem
sa_run = SARunner(problem=knap_problem,
              experiment_name='queens8_sa',
              output_directory=None, # note: specify an output directory to have results saved to disk
              seed=123456,
              iteration_list=2 ** np.arange(11),
              max_attempts=500,
              temperature_list=[0.1, 0.5, 0.75, 1.0, 2.0, 5.0],
              decay_list=[mlrose.GeomDecay])

# the two data frames will contain the results
df_run_stats, df_run_curves = sa_run.run()